# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

In [2]:
%matplotlib qt5

In [10]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/DIADEM")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
cmap='jet'
withPlots = 0
center = 6650
width = 500
nb_ssb = 3

In [66]:
base_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
dic_dir = {"190130" : ["absC"],
           "190131" : ["absAHP12", "absB", "absD", "absJ"],
           "190201" : ["absA", "absAHP20", "absE", "absG", "absM"],
           "190204" : ["absAHP9", "absAHP30", "absAHP45", "absH", "absI", "absK", "absL", "absN", "absO", "absU"]}
processing_path = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15_processing"

In [8]:
dir_A     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190201/absA"
dir_C     = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190128/absC"
dir_AHP9  = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP9"
dir_AHP45 = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190204/absAHP45"

In [43]:
azSlRoElPolFFPolCATR = ser.getAzSlRoElPolFFPolCATR( dir_A )

In [50]:
idxEl = 3
azSlRoElPolFFPolCATR[:,idxEl]

array([-0.24 , -0.24 , -0.24 , -0.191, -0.19 , -0.19 , -0.14 , -0.14 ,
       -0.14 , -0.09 , -0.09 , -0.09 , -0.04 , -0.04 , -0.04 ,  0.01 ,
        0.01 ,  0.01 ,  0.06 ,  0.06 ,  0.06 ,  0.11 ,  0.11 ,  0.11 ,
        0.16 ,  0.16 ,  0.16 ,  0.21 ,  0.21 ,  0.21 ,  0.26 ,  0.26 ,
        0.26 ,  0.31 ,  0.31 ,  0.31 ,  0.36 ,  0.36 ,  0.36 ,  0.41 ,
        0.41 ,  0.41 ,  0.46 ,  0.46 ,  0.46 ,  0.51 ,  0.51 ,  0.51 ,
        0.56 ,  0.56 ,  0.56 ,  0.61 ,  0.61 ,  0.61 ,  0.66 ,  0.66 ,
        0.66 ,  0.71 ,  0.71 ,  0.71 ,  0.76 ,  0.76 ,  0.76 ,  0.81 ,
        0.81 ,  0.81 ,  0.86 ,  0.86 ,  0.86 ,  0.91 ,  0.91 ,  0.91 ,
        0.96 ,  0.96 ,  0.96 ,  1.01 ,  1.01 ,  1.01 ,  1.06 ,  1.06 ,
        1.06 ,  1.11 ,  1.11 ,  1.11 ,  1.16 ,  1.16 ,  1.16 ,  1.21 ,
        1.21 ,  1.21 ,  1.26 ,  1.26 ,  1.26 ,  1.31 ,  1.31 ,  1.31 ,
        1.36 ,  1.36 ,  1.36 ,  1.41 ,  1.41 ,  1.41 ,  1.46 ,  1.46 ,
        1.46 ,  1.51 ,  1.51 ,  1.51 ,  1.56 ,  1.56 ,  1.56 ,  1.61 ,
      

# Fond de chambre

In [46]:
dir_fondDeChambre = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15"
tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre.txt", skiprows=1)
#tmp = np.loadtxt(f"{base_path}/S11_fond_de_chambre_old_without_absA_absC.txt", skiprows=1)
S11_fondDeChambre = tmp[:,1] + 1j * tmp[:,2]

In [47]:
fig, ax = plt.subplots(1,1)
plt.plot( 20 * np.log10( np.abs( np.fft.ifft( S11_fondDeChambre ) ) ), label="fond de chambre" )
plt.grid()
plt.legend()

# Plaque de référence

In [48]:
dir_plaqueDeReference = "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR15/190129/caracterisation/plaque_ref"
freq_all, S11_plaque_ref = ser.getFreqAndS11( dir_plaqueDeReference, nb_ssb )

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

## Initialisation des variables

In [51]:
nb_elev = 41
nb_freq = 2001

elevation        = np.linspace( 0.76 - 1.0, 0.76 + 1.0, nb_elev )
data_S11         = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_td      = np.zeros( (nb_elev, nb_freq), dtype=complex )
data_S11_tg      = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy    = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_td = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_minus_moy_tg = np.zeros( (nb_elev, nb_freq), dtype=complex )
reflectivite     = np.zeros( (nb_elev, nb_freq), dtype=complex )
S11_moy          = np.zeros( nb_freq,            dtype=complex )
S11_moy_square   = np.zeros( nb_freq,            dtype=complex )

## Lecture des données et time gating

### Lecture des données

In [52]:
seq = 0
flag_gate = 1
data_dir = dir_AHP45

for ielev in range(nb_elev):
    freq_all, S11_all = ser.getFreqAndS11( data_dir, nb_ssb, seq0=seq, verbose=0 )
    seq = seq + nb_ssb
    data_S11[ ielev, : ] = S11_all
    
d = np.linspace( 0, 300 / ( freq_all[1] - freq_all[0] ), nb_freq ) * 0.5

### Gate design

scipy.signal.kaiser(M, beta, sym=True)

**SYM** When True (default), generates a symmetric window, for use in filter design. When False, generates a periodic window, for use in spectral analysis

In [53]:
myGate = np.zeros( nb_freq )
myWidth = 500
gateSize = np.where( np.abs( d - center ) < myWidth )[0].shape[0]
myGate[ np.where( np.abs( d - center ) < myWidth ) ] = 1.0
fftMyGate = np.fft.fft( myGate )
print( f"gateSize {gateSize}" )

gateSize 166


In [54]:
fig, (ax0, ax1) = plt.subplots(2,1)
ax0.plot( d, myGate, label="myGate" )
ax0.grid()
ax0.legend()
ax1.plot( 20 * np.log10( np.abs( fftMyGate ) ), label="fft( myGate )" )
ax1.grid()
ax1.legend()

In [55]:
beta = 6
myWindow = np.zeros(nb_freq)
myWindow[0] = 1
myWindow[1:] = np.fft.fftshift( kaiser(nb_freq-1, beta, sym=True) )
modMyWindow = 20 * np.log10( np.abs( myWindow ) )
fftMyGateKaiser = np.fft.fft( myGate ) * myWindow
gateKB = np.fft.ifft( fftMyGateKaiser )

In [56]:
fig, (ax0, ax1) = plt.subplots(2,1)
ax0.plot( myWindow, '.', label='myWindow' )
ax0.grid()
ax0.legend()

ax1.plot( 20 * np.log10( np.abs( fftMyGate ) ), label='fftshift fftMyGate' )
ax1.plot( 20 * np.log10( np.abs( fftMyGateKaiser ) ), label='fftMyGateKaiser' )
ax1.grid()
ax1.legend()

In [57]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20 * np.log10( np.abs( np.real( gateKB ) ) ), '-', label='real gateKB' )
ax.plot( d, 20 * np.log10( np.abs( np.imag( gateKB ) ) ), '-', label='imag gateKB' )
ax.plot( d, 20 * np.log10( np.abs(          gateKB ) ), '.', label='abs gateKB')
plt.grid()
plt.legend()

In [58]:
NKaiser = 200
littleKaiser = kaiser(2*NKaiser, 6, sym=True)
myWindowLittleKaiser = np.zeros(nb_freq)
myWindowLittleKaiser[0] = 1
myWindowLittleKaiser[1:NKaiser+1]      = littleKaiser[NKaiser:]
myWindowLittleKaiser[nb_freq-NKaiser:] = littleKaiser[0:NKaiser]
fftMyGateLittleKaiser = fftMyGate  * myWindowLittleKaiser
gateKB2 = np.fft.ifft( fftMyGateLittleKaiser )

In [59]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20 * np.log10( np.abs( np.real( gateKB2 ) ) ), '-', label='real gateKB2' )
ax.plot( d, 20 * np.log10( np.abs( np.imag( gateKB2 ) ) ), '-', label='imag gateKB2' )
ax.plot( d, 20 * np.log10( np.abs(          gateKB2 ) ), '.', label='abs gateKB2')
plt.grid()
plt.legend()

### Gating

In [60]:
gateLLC = ser.getWindow( d, center, width, id="llc" )

In [61]:
S11_moy     = np.average( data_S11, axis=0 )
S11_moy_td  = np.fft.ifft( S11_moy )
S11_moy_tg_LLC  = S11_moy_td * gateLLC
S11_moy_tg_KB   = S11_moy_td * gateKB

data_S11_td = np.fft.ifft( data_S11, axis=1 )
data_S11_tg_LLC = data_S11_td * gateLLC
data_S11_tg_KB   = data_S11_td * gateKB

In [62]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( data_S11_td[20,:] ) ), label="S11_moy_td" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg_LLC[20,:] ) ), label="S11_moy_tg_LLC" )
ax.plot( d, 20*np.log10( np.abs( data_S11_tg_KB[20,:] ) ),  label="S11_moy_tg_KB" )
ax.legend()
ax.grid()

In [63]:
fig, ax = plt.subplots(1,1)
ax.plot( d, 20*np.log10( np.abs( np.fft.fft( data_S11_td[20,:] ) ) ), label="S11_moy_td" )
ax.plot( d, 20*np.log10( np.abs( np.fft.fft( data_S11_tg_LLC[20,:] ) ) ), label="S11_moy_tg_LLC" )
ax.plot( d, 20*np.log10( np.abs( np.fft.fft( data_S11_tg_KB[20,:] ) ) ),  label="S11_moy_tg_KB" )
ax.legend()
ax.grid()

## Calcul de la réflectivité

In [68]:
gate = gateLLC

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(d, gate, label='gate')
ax.legend()
ax.grid()
fig.savefig("gate.png")

In [ ]:
reflectivite = ser.getReflectivite( data_S11, S11_plaque_ref, S11_fondDeChambre, gate )

## Relecture des données de la réflectivité

In [ ]:
tmp = np.loadtxt(f"{dir_AHP45}/Reflectivite_absAHP45.txt", skiprows=1)
refAbsAHP45 = tmp[:,1:]
refAbsAHP45_f = tmp[:,0]

In [ ]:
vmin = -120
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), refAbsAHP45, vmin=vmin, vmax=vmax, cmap=cmap )

title = f"refAbsAHP45"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [ ]:
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), 20*np.log10(np.abs(reflectivite)).T, 
                    vmin=vmin, vmax=vmax, cmap=cmap )

title = f"réflectivite"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [ ]:
fig, ax = plt.subplots( 1, 1 )
ax.plot( refAbsAHP45[:,0], label="refAbsAHP45" )
ax.plot( 20*np.log10(np.abs(reflectivite)).T[:,0], '.', label="réflectivite" )
ax.legend()
ax.grid()

# Plot

## Coefficient de réflexion

In [ ]:
cmap='jet'

toPlot = 20 * np.log10( np.abs( ( S11_minus_moy_tg ) ) )

valMin = -140
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( freq_all, elevation, toPlot, cmap='jet' )
dia.addColorBar(  im, ax, aspect='auto' )
title = f"Coefficient de réflexion de l'absorbant\n{data_dir}"
ax.set_title(title)
ax.set_ylabel( "Elévation [°]" )
ax.set_xlabel( "Fréquence [GHz]" )

## Réflectivité de l'absorbant

In [ ]:
toPlot = np.zeros( (nb_elev, nb_freq) )
gatebis  = 0.0 * gate
gatebis[ np.where( np.abs( freq_all - 62.5 ) < 12.4 ) ] = 1.0
toPlot = 20 * np.log10( 1e-12 + np.abs( reflectivite * gatebis ) )

valMin = -90
toPlot = 0.5 * ( np.abs( toPlot - valMin ) + toPlot - valMin ) + valMin
valMax = np.amax( toPlot )
valMin = np.amin( toPlot )
print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")
     
vmin = -80
vmax = -0
fig, ax = plt.subplots( 1, 1 )
im = plt.pcolormesh( elevation, np.flip( freq_all ), np.rot90(toPlot), cmap=cmap )

title = f"Réflectivité absorbant\n{data_dir}"
ax.set_title(title)
ax.set_xlabel( "Elévation [°]" )
ax.set_ylabel( "Fréquence [GHz]" )
dia.addColorBar(  im, ax, aspect='auto' )

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(gatebis)

# Tous les absorbants

190130 absC

190131 absAHP12 / absB / absD / absJ

190201 absA / absAHP20 / absE / absG / absM

190204 absAHP9 / absAHP30 / absAHP45 / absH / absI / absK / absL / absN / absO / absU

In [64]:
idx = np.where( np.abs( freq_all - 62.5 ) < 12.4 )
idxAlt = np.where( np.abs( freq_all - 62.5 ) >= 12.4 )
valMin = -90
vmin = -90
vmax = -40
dic_stats = {}

In [65]:
np.save( f"{base_path}/images_190920/data/freq_WR15", freq_all )
np.save( f"{base_path}/images_190920/data/elevation_WR15", elevation )

In [73]:
#plt.ioff()
#plt.ion()

#date = "190130" 
#date = "190131"
#date = "190201"
date = "190204"

idxElevation = 20
suffix = ""
toPlot = np.zeros( (nb_elev, nb_freq) )

for absorbant in dic_dir[date]:

    seq = 0
    flag_gate = 1
    data_dir  = f"{base_path}/{date}/{absorbant}"
    descriptor = f"{absorbant}_{date}"
    
    print(f"======= {absorbant} =======")

    for ielev in range(nb_elev):
        freq_all, S11_all = ser.getFreqAndS11( data_dir, nb_ssb, seq0=seq, verbose=0 )
        seq = seq + 3
        data_S11[ ielev, : ] = S11_all
    
    S11_plaque_ref_tg = np.fft.fft( np.fft.ifft( S11_plaque_ref - S11_fondDeChambre ) * gate )
    tab_freq = freq_all
    S11_minus_moy    = data_S11 - S11_fondDeChambre
    S11_minus_moy_td = np.fft.ifft( S11_minus_moy, axis=1 )
    S11_minus_moy_tg = np.fft.fft( S11_minus_moy_td * gate, axis=1 )

    #=============
    # REFLECTIVITE
    reflectivite = np.zeros( (nb_elev, nb_freq), dtype=complex )
    reflectivite = S11_minus_moy_tg / S11_plaque_ref_tg
    np.save( 
        f"{processing_path}/data/reflectivite{suffix}/{descriptor}{suffix}",
        reflectivite 
    )
    
    # drop lower and higher frequencies values
    toPlot[:,idx]    = 20 * np.log10( 1e-12 + np.abs( reflectivite[:,idx] ) )
    toPlot[:,idxAlt] = -90

    fig, ax = plt.subplots( 1, 1 )
    im = ax.pcolormesh( elevation, np.flip( freq_all ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax )
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel( "Elévation [°]" )
    ax.set_ylabel( "Fréquence [GHz]" )
    dia.addColorBar(  im, ax, aspect='auto' )
    fig.savefig(
        f"{processing_path}/plot/reflectivite{suffix}/reflectivite_{descriptor}{suffix}.png"
    )

    #======
    # STATS
    valMax = np.amax( toPlot )
    valMin = np.amin( toPlot )
    print( f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

    absReflectivite   = np.abs( reflectivite )
    reflectiviteMoy   = np.average( absReflectivite[:,idx] )
    reflectiviteSigma = np.std( absReflectivite[:,idx] )

    reflectiviteMoy_dB   = 20 * np.log10( np.abs( reflectiviteMoy ) )
    reflectiviteSigma_dB = 20 * np.log10( np.abs( reflectiviteSigma ) )
    print( f"moy = {reflectiviteMoy:.6f}, sigma = {reflectiviteSigma:.6f}" )
    print( f"moy = {reflectiviteMoy_dB:.2f} [dB], sigma = {reflectiviteSigma_dB:.2f} [dB]" )
    
    #==========
    # HISTOGRAM
    fig, ax = plt.subplots(1,1)
    ax.hist((absReflectivite[:,idx]).flatten(), 50, label="histogramme (valeurs linéaires)")
    ax.grid()
    title = f"Réflectivité, histogramme [{absorbant}]"
    ax.set_title(title)
    ax.legend()
    fig.savefig(
        f"{processing_path}/plot/hist{suffix}/hist_{descriptor}{suffix}.png"
    )
    
    #=====
    # PLOT
    fig, ax = plt.subplots(1,1)
    ax.plot( freq_all, toPlot[ idxElevation, : ] )
    ax.grid()
    title = f"Réflectivité [{absorbant}]"
    ax.set_title(title)
    ax.set_xlabel("f [GHz]")
    ax.set_ylabel("réflectivité [dB]")
    fig.savefig(
        f"{processing_path}/plot/reflectivite_vs_f{suffix}/reflectivite_vs_f_{descriptor}{suffix}.png"
    )
    
    plt.close("all")
    
    dic_stats[f"{absorbant}"] = (valMax, valMin, reflectiviteMoy, reflectiviteSigma, reflectiviteMoy_dB)

======= absAHP9 =======
valMax = -47.02 valMin = -103.94
moy = 0.001392, sigma = 0.000697
moy = -57.12 [dB], sigma = -63.14 [dB]
======= absAHP30 =======
valMax = -57.90 valMin = -115.86
moy = 0.000275, sigma = 0.000168
moy = -71.23 [dB], sigma = -75.50 [dB]
======= absAHP45 =======
valMax = -59.43 valMin = -128.26
moy = 0.000331, sigma = 0.000168
moy = -69.59 [dB], sigma = -75.47 [dB]
======= absH =======
valMax = -59.46 valMin = -130.06
moy = 0.000298, sigma = 0.000164
moy = -70.51 [dB], sigma = -75.70 [dB]
======= absI =======
valMax = -58.78 valMin = -115.80
moy = 0.000364, sigma = 0.000170
moy = -68.79 [dB], sigma = -75.39 [dB]
======= absK =======
valMax = -57.71 valMin = -126.16
moy = 0.000250, sigma = 0.000156
moy = -72.03 [dB], sigma = -76.15 [dB]
======= absL =======
valMax = -56.22 valMin = -116.11
moy = 0.000377, sigma = 0.000203
moy = -68.47 [dB], sigma = -73.84 [dB]
======= absN =======
valMax = -40.44 valMin = -116.53
moy = 0.000406, sigma = 0.000351
moy = -67.83 [dB], s

In [ ]:
for absorbant in dic_stats.keys():
    stats = dic_stats[absorbant]
    print(f"{absorbant} {stats[0]:.2f} {stats[1]:.2f} {stats[2]:.6f} {stats[3]:.6f} {stats[4]:.2f}")

In [ ]:
elevation.shape